# Ficheux 2020 single-qubit phases
In this tutorial we show how to ideally correct for single-qubit phases accumulated during the implementation of the CPHASE gate between two fluxonia using the Ficheux scheme. The notebook is adapted from the notebook Ficheux 2020.

In [1]:
import numpy as np
import sys
import time
import datetime
import qutip as qtp 
import cmath
import matplotlib.pyplot as plt
from scipy import constants
from scipy.optimize import minimize
import pysqkit
from pysqkit.util.linalg import get_mat_elem, tensor_prod
from pysqkit.util.phys import thermalenergy_to_temperature, temperature_to_thermalenergy
from pysqkit.solvers import solvkit
from typing import List
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
from IPython.display import display, Latex
from itertools import product, combinations

The gates will be based on the following driven Hamiltonian of capacitively coupled fluxonium and transmon

$$H = H_F + H_T + V + H_{drive}$$,

where we have the fluxonium Hamiltonian

$$H_L= 4 E_{C,F} \hat{n}^2_F + \frac{1}{2}E_{L} \hat{\varphi}^2_F - E_{J,F}\cos\left(\hat{\varphi}_F - \varphi_{ext,F}\right), \quad [\hat{\varphi}_F, \hat{n}_F] = i$$,

the transmon Hamiltonian (treated immediately as a Duffing oscillator)

$$H_T =  \hbar \omega_T \hat{b}^{\dagger}\hat{b} + \hbar \frac{\delta_T}{2} \hat{b}^{\dagger} \hat{b}^{\dagger}\hat{b} \hat{b}$$,

where in terms of the transmon's charging energy $E_{C,T}$ and Josephson energy $E_{J, T}$ the transmon's angular velocity $\omega_T$ and anharmonicity $\delta_T$ are

$$ \omega_T = \frac{\sqrt{8 E_{C, T} E_{J, T}}}{\hbar}, \quad \delta_T = - \frac{E_{C,T}}{\hbar} $$.
In addition, the transmon's charge operator can be written as
$$ \hat{n}_T = \frac{i}{2} \biggl( \frac{E_{J, T}}{2 E_{C, T}} \biggr)^{1/4} \bigl( \hat{b}^{\dagger} - \hat{b} \bigr) = \frac{i}{2} \sqrt{\frac{\omega_T}{4 \lvert \delta_T \rvert} }\bigl( \hat{b}^{\dagger} - \hat{b} \bigr).$$

The coupling Hamiltonian is

$$V = J_C  \hat{n}_F \hat{n}_T$$,

and the drive Hamiltonian

$$H_{drive} = 2  f\left( t \right)  \cos \left( \omega_d t + \gamma_d\right) \left( \eta_F  \hat{n}_F + \eta_T \hat{n}_T \right)   \text{  with  } f\left( t \right) = f_0 \dfrac{f\left( t \right)}{f_0}. $$

# Single fluxonia
We take parameters as in Table 1 of the paper. 

In [2]:
# Values in GHz (divided by h)
#Fluxonium A
ec_f = .973
el_f = .457
ej_f = 8.0 
flux_f = 1/2#pi in the cos
n_fock_f = 100

# Transmon parameters: values in GHz divided by 2 \pi 
omega_t = 6.8 #6.0 
delta_t = -0.3
n_fock_t = 3

n_fock_t = 3

In [3]:
levels_f = 5
levels_t = n_fock_t
flx = pysqkit.qubits.Fluxonium('F', ec_f, el_f, ej_f, flux_f, dim_hilbert=n_fock_f)
trans = pysqkit.qubits.SimpleTransmon('T', omega_t, delta_t, dim_hilbert=n_fock_t)
flx.diagonalize_basis(levels_f)

We obtain the energy levels and eigenstates of the two fluxonia

In [4]:
energies_f, eig_states_f = flx.eig_states(levels_f)
energies_t, eig_states_t = trans.eig_states(levels_t)

In [5]:
energies_in_lexico = np.zeros(levels_t*levels_f, dtype=float)
count = 0
for k in range(0, levels_t):
    for m in range(0, levels_f):
        energies_in_lexico[count] = energies_f[m] + energies_t[k]
        count += 1
ascending_to_lexico = np.argsort(energies_in_lexico)

label_converter = ascending_to_lexico
label_converter

array([ 0,  1,  2,  5,  6,  3,  4, 10, 11,  7,  8,  9, 12, 13, 14])

In [6]:
energies_in_lexico = np.zeros(levels_t*levels_f, dtype=float)
count = 0
for k in range(0, levels_t):
    for m in range(0, levels_f):
        energies_in_lexico[count] = energies_f[m] + energies_t[k]
        count += 1
ascending_to_lexico = np.argsort(energies_in_lexico)

label_converter = ascending_to_lexico
label_converter

array([ 0,  1,  2,  5,  6,  3,  4, 10, 11,  7,  8,  9, 12, 13, 14])

In [7]:
# This function allows to return the energy of the "adiabatically continued" coupled levels
def label_to_energies(
    k: int, 
    m: int,
    levels_f: int,
    energies: np.ndarray,  #in ascending order
    label_converter: np.ndarray 
) -> float:
    label = k*levels_f + m
    index = np.where(label_converter == label)[0][0] #index such that converter[index] = label
    return energies[index]

def label_to_states(
    k: int, 
    m: int,
    levels_f: int,
    eig_states: np.ndarray,
    label_converter: np.ndarray 
) -> float:
    label = k*levels_f + m
    index = np.where(label_converter == label)[0][0] #index such that converter[index] = label
    return eig_states[index] 

def energy_ascending_order_to_label(
    k : int,
    levels_f: int,
    label_converter : np.ndarray):
    
    lexico_rank = label_converter[k]
    i = lexico_rank//levels
    j = lexico_rank%levels
    
    return i, j

# Adding drives

In [8]:
flx.add_drive(
    pysqkit.drives.microwave_drive,
    label='cz_drive_f',
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)
trans.add_drive(
    pysqkit.drives.microwave_drive,
    label='cz_drive_t',
    pulse_shape=pysqkit.drives.pulse_shapes.gaussian_top
)

# Coupled fluxonium and transmon

In [9]:
jc = 0.224 #GHz (divided by h)

coupled_flx = trans.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
energies_sys, eigstates_sys = coupled_flx.eig_states() 

coupled_flx_bare = trans.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=0)
energies_bare, eigstates_bare = coupled_flx_bare.eig_states()

# Jump operators and relaxation rates

In [34]:
temperature = 0.020 # K
en_th = temperature_to_thermalenergy(temperature) # kb T/h in GHz
tan_deltac_f = 7.1*1e-6 
tan_deltac_t = 0.5*1e-6 
qdiel_f = 1/tan_deltac_f
qdiel_t = 1/tan_deltac_t

In [35]:
rel_rate_f, exc_rate_f, gamma_f = flx.dielectric_rates(level_k=0, level_m=1, qdiel=qdiel_f, beta=1/en_th)
rel_rate_t, exc_rate_t, gamma_t = trans.dielectric_rates(level_k=0, level_m=1, qdiel=qdiel_t, beta=1/en_th)
t1_f = 1/(2*np.pi)*1/rel_rate_f/1000 # [mu s]
t1_t = 1/(2*np.pi)*1/rel_rate_t/1000 # [mu s]

In [36]:
display(Latex(r'$T_{{1}}^{{a}} $ = {:.3f} $\mu s$'.format(t1_f)))
display(Latex(r'$T_{{1}}^{{b}} $ = {:.3f} $\mu s$'.format(t1_t)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Remark 
The relaxation time of the fluxonium is now very large, keeping the estimated loss tangent fixed as in the Ficheux paper. This is expected since we are now operating at higher $E_J$ and so with a higher barrier. Notice also that now the frequency of the fluxonium is very small

In [37]:
display(Latex(r'$\omega_{{01}}^{{f}}/2 \pi $ = {:.3f} $\mathrm{{GHz}}$'.format(energies_f[1] - energies_f[0])))

<IPython.core.display.Latex object>

This would make the single-qubit gates harder, so maybe it should be increased a bit by increasing $E_L$. Additionally, at the beginning we will use a constant loss tangent. However, that has a dependency with frequency, so we have to be careful when we compare performances with different parameters.

## Drive

In [41]:
eig_states_by_label = []
energies_by_label = []

for i_a in range(levels_t):
    energies_by_label.append([])
    eig_states_by_label.append([])
    for i_b in range(levels_f):
        eig_states_by_label[i_a].append(qtp.Qobj(inpt=  label_to_states(i_a, i_b, levels_f, eigstates_sys, label_converter), 
                   dims=[[levels_t, levels_f], [1, 1]], shape=[levels_t*levels_f, 1]))
        energies_by_label[i_a].append(label_to_states(i_a, i_b, levels_f, energies_sys, label_converter))

In [42]:
n_f = coupled_flx['F'].charge_op(as_qobj=True)
n_t = coupled_flx['T'].charge_op(as_qobj=True)
n_charge = [n_f, n_t]

In [45]:
big_delta = ( label_to_energies(1, 3, levels_f, energies_sys, label_converter)   - \
              label_to_energies(1, 0, levels_f, energies_sys, label_converter) ) - \
            ( label_to_energies(0, 3, levels_f, energies_sys, label_converter)   - \
              label_to_energies(0, 0, levels_f, energies_sys, label_converter) )

display(Latex(r'$\Delta/2\pi $ = {:.3f} MHz'.format(big_delta*1000)))

<IPython.core.display.Latex object>

According to the paper we have :

$$\Omega_{11-21}  = \left| \epsilon_A \langle 11 | \hat{n}_A | 21 \rangle + \epsilon_B \langle 11 | \hat{n}_B | 21 \rangle \right|$$
$$\Omega_{10-20}  = \left| \epsilon_A \langle 10 | \hat{n}_A | 20 \rangle + \epsilon_B \langle 10 | \hat{n}_B | 20 \rangle \right|$$

And then :
$$r = \dfrac{\Omega_{11-21}}{\Omega_{10-20}} \text{  and  } \dfrac{\delta}{\Delta} = \dfrac{r^2 - \sqrt{(r^2-1)^2 + r^2 }} {r^2 - 1}$$ 

With these conventions taken in the paper :
$$\Delta = f_{11-21} - f_{10-20}$$
$$\delta = f_{11-21} - f_d$$

With this $\delta$, we have $\Omega = \Delta$ and a CZ gate is achieved with zero leakage in time exactly $t_{drive} = 1/\Delta$. 

We choose as in the paper to have : $\dfrac{\epsilon_A}{\epsilon_B} = .9$

In [ ]:
ratio_eps = .9

We now match the conditions

In [ ]:
def Omega1(eps_b, ratio_eps):
    eps_a = eps_b*ratio_eps
    eps = [eps_a, eps_b]
    
    rabi_10_20 = np.abs(eps[0] * (eig_states_by_label[1][0].dag() * n_charge[0] * eig_states_by_label[2][0]).full()[0, 0] + \
                              eps[1] * (eig_states_by_label[1][0].dag() * n_charge[1] * eig_states_by_label[2][0]).full()[0, 0] )
    rabi_11_21 = np.abs(eps[0] * (eig_states_by_label[1][1].dag() * n_charge[0] * eig_states_by_label[2][1]).full()[0, 0] + \
                              eps[1] * (eig_states_by_label[1][1].dag() * n_charge[1] * eig_states_by_label[2][1]).full()[0, 0] )

    rabi_ratio = rabi_11_21/rabi_10_20
    ratio_delta = (rabi_ratio**2 - np.sqrt((rabi_ratio**2 - 1)**2 + rabi_ratio**2)) / (rabi_ratio**2 - 1)
    
    big_delta = ( label_to_energies(2, 1, levels, energies_sys, label_converter)   - \
                  label_to_energies(1, 1, levels, energies_sys, label_converter) ) - \
                ( label_to_energies(2, 0, levels, energies_sys, label_converter)   - \
                  label_to_energies(1, 0, levels, energies_sys, label_converter) )
    small_delta = big_delta * ratio_delta
    
    return np.sqrt(rabi_11_21**2 + small_delta**2)

def Omega2(eps_b, ratio_eps):
    eps_a = eps_b*ratio_eps
    eps = [eps_a, eps_b]
        
    rabi_10_20 = np.abs(eps[0] * (eig_states_by_label[1][0].dag() * n_charge[0] * eig_states_by_label[2][0]).full()[0, 0] + \
                              eps[1] * (eig_states_by_label[1][0].dag() * n_charge[1] * eig_states_by_label[2][0]).full()[0, 0] )
    rabi_11_21 = np.abs(eps[0] * (eig_states_by_label[1][1].dag() * n_charge[0] * eig_states_by_label[2][1]).full()[0, 0] + \
                              eps[1] * (eig_states_by_label[1][1].dag() * n_charge[1] * eig_states_by_label[2][1]).full()[0, 0] )

    rabi_ratio = rabi_11_21/rabi_10_20
    ratio_delta = (rabi_ratio**2 - np.sqrt((rabi_ratio**2 - 1)**2 + rabi_ratio**2)) / (rabi_ratio**2 - 1)
    
    big_delta = ( label_to_energies(2, 1, levels, energies_sys, label_converter)   - \
                  label_to_energies(1, 1, levels, energies_sys, label_converter) ) - \
                ( label_to_energies(2, 0, levels, energies_sys, label_converter)   - \
                  label_to_energies(1, 0, levels, energies_sys, label_converter) )
    small_delta = big_delta * ratio_delta
    
    return np.sqrt(rabi_10_20**2 + (small_delta - big_delta)**2)

In [ ]:
rg_e_b = np.linspace(0.01, 1, 1000)

plt.figure(figsize=(8, 6))
plt.plot(rg_e_b, Omega1(rg_e_b, ratio_eps), color = 'blue', label = 'O1')
plt.plot(rg_e_b, Omega2(rg_e_b, ratio_eps), color = 'red', label = 'O2')
plt.plot(rg_e_b, [big_delta]*len(rg_e_b), color = 'orange', label = 'big Delta')

plt.legend()
plt.show()

## Rest of parameters

In [ ]:
eps_b = 0.03634
eps_a = ratio_eps*eps_b
eps = [eps_a, eps_b]

In [ ]:
#we calculate rabi freq amplitudes :

rabi_10_20 = np.abs(eps[0] * (eig_states_by_label[1][0].dag() * n_charge[0] * eig_states_by_label[2][0]).full()[0, 0] + \
                          eps[1] * (eig_states_by_label[1][0].dag() * n_charge[1] * eig_states_by_label[2][0]).full()[0, 0] )

rabi_11_21 = np.abs(eps[0]*(eig_states_by_label[1][1].dag() * n_charge[0] * eig_states_by_label[2][1]).full()[0, 0] + \
                          eps[1]*(eig_states_by_label[1][1].dag() * n_charge[1] * eig_states_by_label[2][1]).full()[0, 0] )

rabi_ratio = rabi_11_21/rabi_10_20
display(Latex(r'$r$ = {:.3f}'.format(rabi_ratio)))

ratio_delta = (rabi_ratio**2 - np.sqrt((rabi_ratio**2 - 1)**2 + rabi_ratio**2)) / (rabi_ratio**2 - 1)
display(Latex(r'$\delta/\Delta$ = {:.3f}'.format(ratio_delta)))

In [ ]:
small_delta = big_delta*ratio_delta

freq_drive = ( label_to_energies(2, 1, levels, energies_sys, label_converter)   - \
               label_to_energies(1, 1, levels, energies_sys, label_converter) )  - small_delta
t_drive = 1/big_delta

display(Latex(r'$\Delta/2\pi $ = {:.3f} MHz'.format(big_delta*1000)))
display(Latex(r'$\delta/2\pi $ = {:.3f} MHz'.format(small_delta*1000)))
display(Latex(r'$\omega_d/2\pi $ = {:.3f} GHz'.format(freq_drive)))

In [ ]:
rabi_freq = big_delta

rabi_period = 1/rabi_freq # [ns]
display(Latex(r'$T_{{Rabi}} $ = {:.3f} ns'.format(rabi_period)))

#  Simulations

In [ ]:
t_rise = 17.6
t_tot = 58.3
# t_plateau = t_tot - 2*t_rise
# simulation setup (noiseless)
pts_per_drive_period = 15

nb_points = int(t_tot*freq_drive*pts_per_drive_period)
tlist = np.linspace(0, t_tot, nb_points)
pulse_drive = np.zeros(nb_points, dtype=float)

interest_levels = [[0,0], [0,1], [1,0], [1,1], [0,2], [1,2], [2,0], [2,1]]
labels = ["$| "+ str(k) + str(m) + " \\rangle $" for k,m in interest_levels]

hamil0 = coupled_flx.hamiltonian(as_qobj=True)
coupled_flx['A'].drives['cz_drive_a'].set_params(phase=0, time=tlist, rise_time=t_rise)
coupled_flx['B'].drives['cz_drive_b'].set_params(phase=0, time=tlist, rise_time=t_rise)

In [ ]:
drive_params = dict(
    cz_drive_a=dict(amp=eps_a, freq=freq_drive),
    cz_drive_b=dict(amp=eps_a, freq=freq_drive)
)

drive_hamils = []
pulses = []

for qubit in coupled_flx:
    if qubit.is_driven:
        for label, drive in qubit.drives.items():
            drive_hamils.append(drive.hamiltonian(as_qobj=True))
            pulses.append(drive.eval_pulse(**drive_params[label]))


In [ ]:
state_in = []
outputs = []
for i in range(0, 2):
    for k in range(0, 2):
        state_in.append(eig_states_by_label[i][k])
for i in range(0, 4):
    outputs.append(solvkit.integrate(tlist*2*np.pi, state_in[i], hamil0, drive_hamils, pulses, [], "mesolve"))

In [ ]:
statein_label = '11'
statein_index = int(statein_label, 2)
rhos_by_label = []
for i in range(len(eig_states_by_label)):
    rhos_by_label.append([])
    for j in range(len(eig_states_by_label[0])):
        rhos_by_label[i].append(eig_states_by_label[i][j]*eig_states_by_label[i][j].dag())  
        
probs = np.zeros([len(labels), nb_points], dtype=float)
for k in range(0, len(labels)):
    i, j = interest_levels[k]
    for m in range(0, nb_points):        
        probs[k, m] = qtp.expect(rhos_by_label[i][j], outputs[statein_index].states[m]) 

In [ ]:
plot_setup = {'lw': 2.0, 'fs': 16, 'fs_text': 14, 'ls': 14}
def plot_probas(
    tlist: np.ndarray, 
    probs: np.ndarray,
    statein_label: str,
    opt: dict
) -> None:
    fig, ax = plt.subplots(int(np.ceil(len(labels)/4)), 4, figsize = (16, 8))
    for k in range(0, len(labels)):
        ax[k//4, k%4].plot(tlist, probs[k, :], linewidth=opt['lw'])

        ax[k//4, k%4].set_ylim(0, 1.0)
        ax[k//4, k%4].set_title(labels[k], fontsize=opt['fs'])
        ax[k//4, k%4].set_ylim(-0.1, 1.1)
        ax[k//4, k%4].axhline(y=0, color='grey', linestyle='--')
        ax[k//4, k%4].axhline(y=1, color='grey', linestyle='--')

        if k%4 != 0:
            ax[k//4, k%4].axes.yaxis.set_ticklabels([])

    fig = plt.gcf()
    fig.suptitle('$\mathrm{Probabilities \ of \ being \ in \ each \ eigenstate \
                 \ as \ a \ function \ of \ time \ [ns] \ with \ initial \ state}$' + 
                 ' $\\vert ' + statein_label + ' \\rangle$', fontsize=opt['fs']) 
    plt.show()

plot_probas(tlist, probs, statein_label, plot_setup)

In [ ]:
u_gate = np.zeros([4, 4], dtype=complex)
for i in range(0, 4):
    for k in range(0, 4):
        u_gate[i, k] = (state_in[i].dag()*outputs[k].states[nb_points - 1])[0, 0]
u_ideal_phases = np.zeros([4, 4], dtype=complex)
for i in range(0, 2):
    for k in range(0, 2):
        n = i*2 + k
        u_ideal_phases[n, n] = np.exp(-1j*2*np.pi*energies_by_label[i][k]*t_tot)

#taking out phase associated with 00 from numerical simulations
u_ideal_phases = np.conj(u_gate[0, 0])*u_ideal_phases
u_gate = np.conj(u_gate[0, 0])*u_gate 

We now detail how to obtain the single-qubit gates in order to bring the (ideal) gate to the CZ form
$$
\mathrm{CZ} = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & -1
\end{pmatrix}
$$
The Ficheux protocol we have just simulated realizes (ideally) a unitary 
$$
U =  \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & e^{i \phi_{01}} & 0 & 0 \\
0 & 0 & e^{i \phi_{10}} & 0 \\
0 & 0 & 0 & e^{i \phi_{11}}
\end{pmatrix}
$$
where the phases satisfy the condition
$$
\phi_{11} - \phi_{01} - \phi_{10} = \pi \mod 2 \pi.
$$
Let us define a generic single-qubit phase gate
$$
P(\phi) = \begin{pmatrix}
1 & 0 \\
0 & e^{i \phi}
\end{pmatrix}.
$$
The single-qubit gates to be applied are $P(-\phi_{01})$ on the second qubit and $P(-\phi_{10})$ on the first qubit (to see it more clearly apply them sequentially). In fact, one can check that 
$$
P(-\phi_{10}) \otimes P(-\phi_{01}) U = \begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & e^{i (\phi_{11} - \phi_{10} - \phi_{01})}
\end{pmatrix},
$$
which given the previous condition on the phases is equal to the CZ gate.


I remark that this way of obtaining the phases assumes the ideality of the model. This does not mean that is the optimal way. For instance I noticed that (quite surprisingly) the ideal phase acquired by $\lvert 00 \rangle$ is not the same as the one acquired in the simulations, but there is a small deviation

In [ ]:
display(Latex(r'$\phi_{{00}}^{{\mathrm{{ideal}}}} $ = {:.3f} '.format(cmath.phase(u_ideal_phases[0, 0]))))
display(Latex(r'$\phi_{{00}} $ = {:.3f} '.format(cmath.phase(u_gate[0, 0]))))

I interpret this as an effect of the drive that is dressing the states. This effect might or might not be correctable by single-qubit gates, but they must be optimized and this is not included in the previous ideal analysis.

In [ ]:
def single_qubit_correction(
    u_gate: np.ndarray
) -> np.ndarray:
    phi_10 = cmath.phase(u_gate[2, 2])
    phi_01 = cmath.phase(u_gate[1, 1])
    p_phi10 = np.array([[1, 0], [0, np.exp(-1j*phi_10)]])
    p_phi01 = np.array([[1, 0], [0, np.exp(-1j*phi_01)]])
    return tensor_prod([p_phi10, p_phi01])

In [ ]:
cz = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]])
u_correction = single_qubit_correction(u_gate)
u_ideal = qtp.Qobj(u_correction.conj().T.dot(cz))
sup_u_ideal = qtp.to_super(u_ideal)